In [18]:
import pandas as pd
import json

# JSON 파일 읽기
with open('senti_result2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# JSON 데이터를 데이터프레임으로 변환
df = pd.DataFrame(data)

# 수강후기가 있는 경우만 선택하여 처리
df_with_review = df[df['review'] != '수강후기가 없습니다.']

# course_id 별로 review와 해당 review에 대응하는 result 묶기
grouped = df_with_review.groupby('course_id').apply(lambda x: [{'review': r, 'result': res} for r, res in zip(x['review'], x['result'])]).reset_index(name='reviews')

# course_id를 key로 하는 JSON 객체 생성
grouped_json = {}
for index, row in grouped.iterrows():
    grouped_json[str(row['course_id'])] = row['reviews']

# JSON 파일로 저장
with open('grouped_reviews.json', 'w', encoding='utf-8') as file:
    json.dump(grouped_json, file, ensure_ascii=False, indent=4)


C:\Users\user\AppData\Local\Temp\ipykernel_17208\1028846666.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df_with_review.groupby('course_id').apply(lambda x: [{'review': r, 'result': res} for r, res in zip(x['review'], x['result'])]).reset_index(name='reviews')


In [8]:
# 'course_name'과 'review' 컬럼 선택
selected_columns = df[['course_name', 'course_id', 'result','review']]

# 선택된 컬럼 출력
print(selected_columns)

                                           course_name  course_id  result  \
0                                       소형굴착기 자격증 취득과정          1     1.0   
1                                       소형굴착기 자격증 취득과정          1     1.0   
2                                       소형굴착기 자격증 취득과정          1     1.0   
3                                       소형굴착기 자격증 취득과정          1     1.0   
4           개인회생파산 서류작성 실무(주말반 또는 평일반, 회생 실무관련 사이트 활용)          2     0.0   
...                                                ...        ...     ...   
112927                   한식조리기능사&양식조리기능사 자격증 취득(이론+실기)      17928     1.0   
112928                   한식조리기능사&양식조리기능사 자격증 취득(이론+실기)      17928     1.0   
112929                          디지털 전환 시대의 정보보호 전문가 과정      17929     0.0   
112930  NX(UG) NASTRAN CAE(구조/진동/모션/다물체/열전달/열응력/유동) 해석      17930     0.0   
112931                        3D 형상모델링 Inventor(인벤터)_A      17931     0.0   

                                                review  
0       열정적인 강사님과 

In [10]:
import pandas as pd
import numpy as np
import json

# JSON 파일 읽기
with open('ps_가중치2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# JSON 데이터를 데이터프레임으로 변환
df = pd.DataFrame(data)

# 각 열의 최소값과 최대값 계산
average_result_min = df['average_result'].min()
average_result_max = df['average_result'].max()
positive_count_min = df['positive_count'].min()
positive_count_max = df['positive_count'].max()

# 최소값과 최대값의 차이를 이용하여 중요도를 평가
average_result_range = average_result_max - average_result_min
positive_count_range = positive_count_max - positive_count_min

# average_result가 더 중요하다고 가정하여 상대적 중요도 평가
average_result_importance = 4  # 예: average_result가 3배 더 중요하다고 가정
positive_count_importance = 1

# 쌍대 비교 행렬 생성
matrix = np.array([[1, average_result_importance],
                   [1/average_result_importance, 1]])

# 고유 벡터 계산
eigenvalues, eigenvectors = np.linalg.eig(matrix)

# 최대 고유값과 대응하는 고유 벡터 선택
max_eigenvalue = np.max(eigenvalues)
max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]

# 가중치 계산 (정규화)
weights = max_eigenvector / np.sum(max_eigenvector)
weights = weights.real  # 실수부만 사용

# AHP에서 도출된 가중치
average_result_weight = weights[0]
positive_count_weight = weights[1]

# 가중치 출력 (확인용)
print(f"average_result_weight: {average_result_weight}, positive_count_weight: {positive_count_weight}")

# positive_count 정규화
df['normalized_positive_count'] = (df['positive_count'] - df['positive_count'].min()) / (df['positive_count'].max() - df['positive_count'].min())

# 가중치 결과 계산
df['weighted_score'] = (df['average_result'] * average_result_weight) + (df['normalized_positive_count'] * positive_count_weight)

# 가중치 점수를 100점 만점으로 환산
df['final_weighted_score'] = (df['weighted_score'] * 100).round(2)

# 결과 데이터프레임 준비
result_df = df[['course_id',  'course_name', 'ncs_num', 'edu_institute', 'average_result', 'positive_count', 'star', 'emp', 'final_weighted_score', 'all_reviews']]

# 결과를 CSV 파일로 저장
result_df.to_csv('./mnt/data/weighted_results.csv', index=False, encoding='utf-8')

# 결과를 JSON 파일로 저장
result_df.to_json('./mnt/data/weighted_results.json', orient='records', force_ascii=False)

result_df


average_result_weight: 0.6315789473684209, positive_count_weight: 0.21052631578947376, star_weight: 0.15789473684210528


,course_id,course_name,ncs_num,edu_institute,average_result,positive_count,star,emp,final_weighted_score,all_reviews
0,1,소형굴착기 자격증 취득과정,14070106,대경녹색기술직업전문학교,1.000,4,92.9,,77.96,열정적인 강사님과 함께 해서 잘 배울수 있었고 시간 일동안 소형굴삭기 다배우기에는...
1,3,"유튜브(Youtube) 크리에이터(프리미어,에프터이펙트)",08030204,더조은컴퓨터아트학원,1.000,3,89.8,,77.31,처음에는 어려워서 과연 잘한 선택이였을까 싶을정도로 어렵게만 느껴졌는데 두번 수업 ...
2,4,한과조리(떡제조기능사 이론/실기),13010101,수영음식나라조리학원,1.000,11,97.1,43.4,79.28,어려울 줄만 알았던 떡을 쉽게 배울수 있는 수업이였다 떡이 앞으로 인기도 많고 전...
3,6,직업상담실무-직업심리검사 활용 & 상담사례관리,07020101,종로여성인력개발센터,1.000,5,100.0,56.1,79.34,자격증만 취득하고 실제 업무가 막막했는데 실무에 대한 자신감을 가져다 줄 수 있는 ...
4,7,집합건물 경리실무자 양성,02030201,한국직업능력교육원 안산,0.923,24,97.3,71.7,75.49,열정있는 강사님이 정말 좋았지만 짧은교육시간에 함축적으로 수업을 진행한 점이 많이 ...
...,...,...,...,...,...,...,...,...,...,...
7879,17838,"산업안전기사, 산업기사 필기 취득",23060102,(재단법인)서울호서직업전문학교,1.000,5,100.0,,79.34,아주 만족스런 강의였습니다 몰랐던 내용까지 상세히 더 깊이 알려주셔서 의미있는 시간...
7880,17859,애견수제간식(펫푸드),13010101,궁전요리제빵커피직업전문학교(효자점),1.000,64,96.3,47.7,83.33,유익한 수강이였다 우리집 반려견이 수제간식을 너무 맛있게 먹어 익산에서 전주로 다닌...
7881,17883,가스설비시공&취급관리(기업맞춤형),14030302,동아기술직업전문학교,0.917,11,97.1,54.5,74.04,처음에는 아버지 친구 분의 권유로 시작했는데 이곳에서 차분하게 준비하면서 앞으로 어...
7882,17892,자동차정비 산업기사 자격증 대비과정(필기),15060302,홍익자동차직업전문학교,1.000,9,95.9,44.4,78.90,강사님이 친절하게 가르쳐주셔서 많은 도움을 받았습니다 감사합니다 수업듣고 이해안될때...


In [12]:
import pandas as pd
import numpy as np
import json

# JSON 파일 읽기
with open('ps_가중치2.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# JSON 데이터를 데이터프레임으로 변환
df = pd.DataFrame(data)

# 각 열의 최소값과 최대값 계산
average_result_min = df['average_result'].min()
average_result_max = df['average_result'].max()
positive_count_min = df['positive_count'].min()
positive_count_max = df['positive_count'].max()
star_min = df['star'].min()
star_max = df['star'].max()

# 최소값과 최대값의 차이를 이용하여 중요도를 평가
average_result_range = average_result_max - average_result_min
positive_count_range = positive_count_max - positive_count_min
star_range = star_max - star_min

# 각 변수의 상대적 중요도 평가
average_result_importance = 4  # 예: average_result가 4배 더 중요하다고 가정
positive_count_importance = 1
star_importance = 4

# 쌍대 비교 행렬 생성
matrix = np.array([[1, average_result_importance, star_importance],
                   [1/average_result_importance, 1, star_importance / average_result_importance],
                   [1/star_importance, average_result_importance / star_importance, 1]])

# 고유 벡터 계산
eigenvalues, eigenvectors = np.linalg.eig(matrix)

# 최대 고유값과 대응하는 고유 벡터 선택
max_eigenvalue = np.max(eigenvalues)
max_eigenvector = eigenvectors[:, np.argmax(eigenvalues)]

# 가중치 계산 (정규화)
weights = max_eigenvector / np.sum(max_eigenvector)
weights = weights.real  # 실수부만 사용

# AHP에서 도출된 가중치
average_result_weight = weights[0]
positive_count_weight = weights[1]
star_weight = weights[2]

# 가중치 출력 (확인용)
print(f"average_result_weight: {average_result_weight}, positive_count_weight: {positive_count_weight}, star_weight: {star_weight}")

# positive_count와 star 정규화
df['normalized_positive_count'] = (df['positive_count'] - df['positive_count'].min()) / (df['positive_count'].max() - df['positive_count'].min())
df['normalized_star'] = (df['star'] - df['star'].min()) / (df['star'].max() - df['star'].min())

# 가중치 결과 계산
df['weighted_score'] = (df['average_result'] * average_result_weight) + \
                       (df['normalized_positive_count'] * positive_count_weight) + \
                       (df['normalized_star'] * star_weight)

# 가중치 점수를 100점 만점으로 환산
df['final_weighted_score'] = (df['weighted_score'] * 100).round(2)

# 결과 데이터프레임 준비
result_df = df[['course_id', 'course_name', 'ncs_num', 'edu_institute', 'average_result', 'positive_count', 'star', 'emp', 'final_weighted_score', 'all_reviews']]

# 결과를 CSV 파일로 저장
result_df.to_csv('./mnt/data/weighted_results.csv', index=False, encoding='utf-8')

# 결과를 JSON 파일로 저장
result_df.to_json('./mnt/data/weighted_results.json', orient='records', force_ascii=False)

result_df


average_result_weight: 0.6666666666666669, positive_count_weight: 0.1666666666666666, star_weight: 0.16666666666666663


,course_id,course_name,ncs_num,edu_institute,average_result,positive_count,star,emp,final_weighted_score,all_reviews
0,1,소형굴착기 자격증 취득과정,14070106,대경녹색기술직업전문학교,1.000,4,92.9,,82.20,열정적인 강사님과 함께 해서 잘 배울수 있었고 시간 일동안 소형굴삭기 다배우기에는...
1,3,"유튜브(Youtube) 크리에이터(프리미어,에프터이펙트)",08030204,더조은컴퓨터아트학원,1.000,3,89.8,,81.54,처음에는 어려워서 과연 잘한 선택이였을까 싶을정도로 어렵게만 느껴졌는데 두번 수업 ...
2,4,한과조리(떡제조기능사 이론/실기),13010101,수영음식나라조리학원,1.000,11,97.1,43.4,83.46,어려울 줄만 알았던 떡을 쉽게 배울수 있는 수업이였다 떡이 앞으로 인기도 많고 전...
3,6,직업상담실무-직업심리검사 활용 & 상담사례관리,07020101,종로여성인력개발센터,1.000,5,100.0,56.1,83.65,자격증만 취득하고 실제 업무가 막막했는데 실무에 대한 자신감을 가져다 줄 수 있는 ...
4,7,집합건물 경리실무자 양성,02030201,한국직업능력교육원 안산,0.923,24,97.3,71.7,79.18,열정있는 강사님이 정말 좋았지만 짧은교육시간에 함축적으로 수업을 진행한 점이 많이 ...
...,...,...,...,...,...,...,...,...,...,...
7879,17838,"산업안전기사, 산업기사 필기 취득",23060102,(재단법인)서울호서직업전문학교,1.000,5,100.0,,83.65,아주 만족스런 강의였습니다 몰랐던 내용까지 상세히 더 깊이 알려주셔서 의미있는 시간...
7880,17859,애견수제간식(펫푸드),13010101,궁전요리제빵커피직업전문학교(효자점),1.000,64,96.3,47.7,86.62,유익한 수강이였다 우리집 반려견이 수제간식을 너무 맛있게 먹어 익산에서 전주로 다닌...
7881,17883,가스설비시공&취급관리(기업맞춤형),14030302,동아기술직업전문학교,0.917,11,97.1,54.5,77.93,처음에는 아버지 친구 분의 권유로 시작했는데 이곳에서 차분하게 준비하면서 앞으로 어...
7882,17892,자동차정비 산업기사 자격증 대비과정(필기),15060302,홍익자동차직업전문학교,1.000,9,95.9,44.4,83.10,강사님이 친절하게 가르쳐주셔서 많은 도움을 받았습니다 감사합니다 수업듣고 이해안될때...


In [11]:
import pandas as pd
import numpy as np
import json

# JSON 파일 읽기 및 데이터프레임 변환
with open('./mnt/data/weighted_results.json', 'r', encoding='utf-8') as file:
    result_df = pd.read_json(file)

# emp 열의 빈 값을 NaN으로 대체
result_df['emp'] = result_df['emp'].replace('', np.nan).astype(float)

# edu_institute별로 그룹화하여 각 컬럼의 평균 계산 (NaN 값 제외)
grouped_df = result_df.groupby('edu_institute').agg({
    'average_result': lambda x: x.dropna().mean(),
    'positive_count': lambda x: x.dropna().mean(),
    'star': lambda x: x.dropna().mean(),
    'emp': lambda x: x.dropna().mean(),
    'final_weighted_score': lambda x: x.dropna().mean(),
    'all_reviews': lambda x: ' '.join(x.dropna())  # all_review 컬럼을 텍스트로 통합
}).reset_index()



grouped_df = grouped_df.round({
    'average_result': 2,
    'positive_count': 2,
    'star': 2,
    'emp': 2,
    'final_weighted_score': 2
})

cleaned_df = grouped_df.dropna()

# 인덱스 추가
cleaned_df.reset_index(drop=True, inplace=True)
cleaned_df.index.name = 'index'

# 결과를 CSV 파일로 저장 (인덱스를 포함하여 저장)
cleaned_df.to_csv('./mnt/data/weight_result.csv', index=True, encoding='utf-8')

# 결과를 JSON 파일로 저장 (인덱스를 포함하여 저장)
cleaned_df.to_json('./mnt/data/weight_result.json', orient='records', force_ascii=False)
cleaned_df


,edu_institute,average_result,positive_count,star,emp,final_weighted_score,all_reviews
index,,,,,,,
0,(사)CH부설제주직업전문학원,0.74,6.40,83.50,39.90,60.03,일주일마다 조를 바꿔가면 다른 수강생분들과 어우러져 수업을 들을 수 있어서 좋았어요...
1,(사)도시생명네트워크부설대동요양보호사교육원,1.00,14.00,97.60,54.70,79.61,원장님이하 강사님들께서 친절하게 잘 가르켜 주셔서 자격증을 취득하였고 다른분들에게 ...
2,(사)충주YWCA요양보호사교육원,0.91,20.00,95.80,55.70,74.01,여러가지 몰랐던것을 많이배웠다 수강을 통해 치매에관하여 알게되었고 일반생활에서 많은...
3,(사)한국디지털컨버전스협회,0.92,1.00,78.25,39.40,70.16,일러스트 어려운데 열심희 공부헤야겟네여 수업내용이 매우 유용하게 잘 구성되었습니다 ...
4,(사)한국수상레저안전연합회 부산지부,1.00,1.00,92.90,44.00,77.72,훈련과정이 전혀 지루하지않고 재밌고 매우 유익하였다
...,...,...,...,...,...,...,...
1569,흙바라기도예공방학원,1.00,9.67,98.03,31.70,79.35,도예 물레에 대해 적합한 지식과 실기를 전반적으로 익히는 데 도움되었습니다 자기계발...
1570,희망가득요양보호사교육원,1.00,30.67,99.30,30.30,81.25,직장다니면서 주말반 요양보호사반을 들었는데요 학원도 좋고 실습도 같은 건물에서 해서...
1571,희망요양보호사교육원,0.90,24.00,88.10,24.65,72.66,간호학 강의는 좋았습니다 다만 그외 강의에서 혐오발언이나 자신 자식자랑 등이 듣기 ...
